In [1]:
import pandas as pd
import os
import sys 
import numpy as np
from functools import reduce
import itertools
from ast import literal_eval #converts object list to list of strings
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import statsmodels.api as sm
from statsmodels.formula.api import ols

# this points to a Python file with the function country_mappings (not used)
from combine_country_regions import country_mappings

# not great practice, but this removes warnings from the output
import warnings
warnings.filterwarnings("ignore")

# display settings so I can see more on the screen
desired_width=1000
pd.set_option('display.width', desired_width)
pd.set_option('display.max_columns',10)
pd.options.display.max_rows = 50


ModuleNotFoundError: No module named 'combine_country_regions'

In [ ]:

#############################################################
# set this to point to your folder or create a new folder,
# (in my case my computer is called jpark and I called the folder trade_warning) 
#############################################################
os.chdir(r'C:\Users\jpark\VisualStudio\Simpsons_BACI\\')


In [ ]:

# points to country codes as defined by BACI
COUNTRY_CODES = r"C:\Users\jpark\Downloads\BACI_HS22_V202401b\country_codes_V202401b.csv"
# point to product codes
PRODUCT_DESCRIPTION = r"C:\Users\jpark\Downloads\BACI_HS22_V202401b\product_codes_HS22_V202401b.csv"
# add region data, might be better sources
ADD_REGIONS = r"data\\iso_countries_regions.csv"
# add short HS2 description (could be better descriptions)
SHORT_CODES = r"src\baci\data\hs6twodigits.csv"
# add long product description
LONG_DESCRIPTION = r"C:\Users\jpark\Downloads\BACI_HS22_V202401b\product_codes_HS22_V202401b.csv"
# add gdp data
GDP_DATA = r"data\\global_gdp.csv"
# strategic goods
STRATEGIC_GOODS = r"src\\pdf_extractor\\strategicProducts.csv"
# wgi values
WGI_VALUES = r"src\baci\various_codes\WGI_iso3.csv"
# location of BACI unpacked data, note(!) last part of string, I keep the year 
# as a variable, so I can write, for instance:
j = 2022
BACI_DATA = r"C:\\Users\\jpark\\Downloads\\BACI_HS92_V202401b\BACI_HS92_Y"
BACI_DATA + str(j) + "_V202401.csv"

class baci:
    '''baci class contains the methods to load baci data and add characteristics such as geographic and strategic'''
    def readindata(self, bacidata, verbose = False, tmp_save = True) -> pd.DataFrame:
        '''main method to read in baci data'''
        df1 = pd.read_csv(bacidata, usecols=['t','i','j','k','v','q'], 
                          dtype= {'t': 'int64',
                                  'i': 'int64', 
                                  'j': 'int64', 
                                  'k': 'object',
                                  'v': 'float64',
                                  'q': 'object'}
                          )

        # This is too complicated, but '   NA' should be converted to float
        df1['q'] = df1['q'].apply(lambda x: x.strip()) # remove spaces in data
        df1['q'].replace('NA', np.NaN, inplace=True)   # np.NaN is different than string NaN
        df1['q'] = df1['q'].astype(float)

        # rename columns to make them meaningful to humans
        df1.rename(columns={'t': 'Year', 'i': 'Exporter', 'j': 'Importer', 'k': 'Product', 'v': 'Value', 'q': 'Quantity'}, inplace=True)

        # replace number with name of country *exporter* 
        iso1 = pd.read_csv(COUNTRY_CODES, usecols=['country_code', 'country_iso3'])
        df1 = df1.merge(iso1, left_on="Exporter", right_on="country_code", how="left")
        df1.drop(columns=['country_code', 'Exporter'], inplace = True)
        df1.rename(columns={"country_iso3": "Exporter"}, inplace=True)
    
        # replace number with name of country *importer*
        df1 = df1.merge(iso1, left_on="Importer", right_on="country_code", how="left")
        df1.drop(columns=['country_code', 'Importer'], inplace = True)
        df1.rename(columns={"country_iso3": "Importer"}, inplace=True)

        # 2015 has some strange data, take only Values greater than 10.00, otherwise number of exporting countries in 2015 is an outlier
        df1 = df1[df1['Value'] > 10.00]

        # if verbose is True, this will print out
        if verbose:
            hcodes = [str(x)[0:2] for x in df1["Product"]]
            print(set(hcodes))
            print(len(set(hcodes)))

        # make product code and int, otherwise its an object which can be confusing
        df1['Product'] = df1['Product'].astype(int)    

        return df1
    
    def addprodcode(self, data):
        '''add the product description if needed'''
        # add product_codes
        prodcodes = pd.read_csv(PRODUCT_DESCRIPTION, usecols=['code', 'description'])
        # product '9999AA' appears to be a filler--empty
        mask = prodcodes['code'] == '9999AA'
        prodcodes = prodcodes[~mask]
        # I love merges, note its a left merge, I want all baci data to have a code, but dont care for product codes without products.
        data = data.merge(prodcodes, left_on = "Product", right_on = "code", how = "left")
        
        return data
    
    def addlongdescription(self, data):
        '''Add product product description based on codes'''
        localdata = data.copy()
        longdesc = pd.read_csv(LONG_DESCRIPTION, dtype = str)

        # this is necessary because codes 1:9 should be 01:09
        localdata.loc[:, 'Product'] = ["0" + x if len(x) == 5 else x for x in localdata['Product'].astype(str)]

        longdesc.rename(columns = {"code": "isocode"}, inplace=True)
        longproddesc = localdata.merge(longdesc, left_on="Product", right_on="isocode", how = 'left', suffixes = ['x', 'y'])
       
        r1 = localdata.shape[0]
        r2 = longproddesc.shape[0]
        assert r1 == r2

        return longproddesc
    
    def add_gdp(self, data, GDP, year):
        '''Join GDP to data'''

        ### join GDP to data
        
        # Exporters
        gdp = GDP[GDP.index == year]
        gdp = gdp.T
        gdp['Exporter_gdp'] = gdp.index
        
        gdp.rename(columns={year: year + "_gdp_Exporter"}, inplace=True)

        dataj = data.merge(gdp, left_on = "Exporter", right_on = "Exporter_gdp")
        dataj[year + '_gdp_Exporter'] = dataj[year + '_gdp_Exporter']/1e+6
        
        # Importers
        gdp = GDP[GDP.index == year]
        gdp = gdp.T
        gdp['Importer_gdp'] = gdp.index
        gdp.rename(columns={year: year + '_gdp_Importer'}, inplace=True)

        data = dataj.merge(gdp, left_on = "Importer", right_on = "Importer_gdp")
       
        data.drop(columns = ["Exporter_gdp", "Importer_gdp"], inplace=True)

        return data
         


In [ ]:

def GDPData():
    '''should alway be run, need to move to BACI class'''
    # https://data.worldbank.org/indicator/NY.GDP.MKTP.CD?end=2022&start=1960&view=chart
    # Taiwan comes from IMF data, added by hand. https://www.imf.org/external/datamapper/NGDPD@WEO/OEMDC/ADVEC/WEOWORLD
    
    data = pd.read_csv(GDP_DATA, index_col=[1], skiprows=4)
    data = data.drop(columns=['Country Name', 'Indicator Code', 'Indicator Name'])
    data = data.T
    return data
GDP = GDPData()

In [ ]:

# #############################################################
# # INITIALIZE object, needs to be run to create a BACI object instance
bc1 = baci()
# #############################################################

# test data
j = 2022
bacidata = BACI_DATA + str(j) + "_V202401b.csv"
test_data = bc1.readindata(bacidata, verbose = False, tmp_save = False)
ategicgoodExportingImportingregions(data, impexp = "Importer")
# strategicproducts = bc1.typesofstrategicgoods(data)

def barchartstrategicprod(strategicproducts):
    ax = strategicproducts[['Value']].plot.barh(stacked=False,  rot=0, cmap='tab20', figsize=(10, 7))
    ax.legend('best')
    plt.tight_layout()
    ax.set_title("Value of trade strategic goods by HS6 2-digit category)")
    plt.show()
    plt.savefig("output\ValueoftradestrategicgoodHS6",bbox_inches='tight')
#barchartstrategicprod(strategicproducts)